# モデルチューニング

In [1]:
# Google Driveと接続を行います。これを行うことで、Driveにあるデータにアクセスできるようになります。
# 下記セルを実行すると、Googleアカウントのログインを求められますのでログインしてください。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
import numpy as np
import pandas as pd
import os
import pickle
import gc

# 分布確認  anacondaでは、一先ず、コメントにする。
#import pandas_profiling as pdp

# 可視化
import matplotlib.pyplot as plt

# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

# バリデーション
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# 評価指標
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

# モデリング: lightgbm
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

# matplotilbで日本語表示したい場合はこれをinstallしてインポートする
!pip install japanize-matplotlib
import japanize_matplotlib
%matplotlib inline


### ニューラルネットワークの適用：①全結合層のみのネットワークモデル
#### ライブラリのインポート

In [38]:
pip install tensorflow

In [39]:
import tensorflow as tf  ### TensorFlowライブラリをインポート

from tensorflow.keras.models import Sequential, Model  ### KerasのSequentialモデルとModelクラスをインポート
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization  ### KerasのInput、Dense、Dropout、BatchNormalizationレイヤーをインポート
from tensorflow.keras.layers import Embedding, Flatten, Concatenate  ### KerasのEmbedding、Flatten、Concatenateレイヤーをインポート
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler  ### KerasのEarlyStopping、ModelCheckpoint、ReduceLROnPlateau、LearningRateSchedulerをインポート
from tensorflow.keras.optimizers import Adam, SGD  ### KerasのAdam、SGDオプティマイザをインポート
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau  ###anacondaで追加処理

#### tensorflowの再現性のためのシード指定

In [40]:
import os   ###anacondaで追加処理
import numpy as np ###anacondaで追加処理


def seed_everything(seed):
    #import random 　anacondaでは不使用
    #random.seed(seed)　anacondaでは不使用
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )

    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    #以下に変更　tf.compat.v1.keras.backend.set_session(sess)
    tf.keras.backend # tf.keras.backendに変更

seed_everything(seed=123)    # 関数を呼び出し、シード値123を使用して乱数生成器の設定を行う




#### ファイルの読み込みとデータセット作成

In [63]:
## ファイル読み込み
df_train = pd.read_csv("Bank Churn/train.csv")

#特徴量エンジニアリング
df_train["NumOfProducts_/_Age"] = df_train["NumOfProducts"] / df_train["Age"]

# BalanceとNumOfProductsの四則計算の特徴量作成　（数値変数×数値変数）
df_train["Balance_-_NumOfProducts"] = df_train["Balance"] - df_train["NumOfProducts"]

df_train["mean_NumOfProducts_by_HasCrCard"] = df_train.groupby("NumOfProducts")["HasCrCard"].transform("mean")

# データセット作成
x_train = df_train[["IsActiveMember","EstimatedSalary","Age","NumOfProducts","NumOfProducts_/_Age","Balance_-_NumOfProducts","mean_NumOfProducts_by_HasCrCard"]]
y_train = df_train[["Exited"]]


#### 数値データの正規化

In [64]:
### "下の各特徴量の列の値を最小値で引き、その結果を"下の特徴量の列の最大値と最小値の差で割ることで、最小-最大スケーリングを実行
x_train["IsActiveMember"] = (x_train["IsActiveMember"] - x_train["IsActiveMember"].min()) / (x_train["IsActiveMember"].max() - x_train["IsActiveMember"].min())

x_train["EstimatedSalary"] = (x_train["EstimatedSalary"] - x_train["EstimatedSalary"].min()) / (x_train["EstimatedSalary"].max() - x_train["EstimatedSalary"].min())

x_train["Age"] = (x_train["Age"] - x_train["Age"].min()) / (x_train["Age"].max() - x_train["Age"].min())

x_train["NumOfProducts"] = (x_train["NumOfProducts"] - x_train["NumOfProducts"].min()) / (x_train["NumOfProducts"].max() - x_train["NumOfProducts"].min())

x_train["NumOfProducts_/_Age"] = (x_train["NumOfProducts_/_Age"] - x_train["NumOfProducts_/_Age"].min()) / (x_train["NumOfProducts_/_Age"].max() - x_train["NumOfProducts_/_Age"].min())

x_train["Balance_-_NumOfProducts"] = (x_train["Balance_-_NumOfProducts"] - x_train["Balance_-_NumOfProducts"].min()) / (x_train["Balance_-_NumOfProducts"].max() - x_train["Balance_-_NumOfProducts"].min())

x_train["mean_NumOfProducts_by_HasCrCard"] = (x_train["mean_NumOfProducts_by_HasCrCard"] - x_train["mean_NumOfProducts_by_HasCrCard"].min()) / (x_train["mean_NumOfProducts_by_HasCrCard"].max() - x_train["mean_NumOfProducts_by_HasCrCard"].min())




####  学習データと検証データの分割

In [65]:
x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train, random_state=123)
print(x_tr.shape, x_va.shape, y_tr.shape, y_va.shape)

(132027, 7) (33007, 7) (132027, 1) (33007, 1)


#### モデル定義

In [62]:
def create_model():  ### モデルを作成する関数の定義を開始

    input_num = Input(shape=(8,))  ### 入力層を定義し、入力データの形状を指定

    x_num = Dense(10, activation="relu")(input_num)  ### 10個のニューロンを持つ全結合層を追加し、ReLU活性化関数を適用

    x_num = BatchNormalization()(x_num)  ### バッチ正規化を適用

    x_num = Dropout(0.3)(x_num)  ### ドロップアウトを適用

    x_num = Dense(10, activation="relu")(x_num)  ### 10個のニューロンを持つ全結合層を追加し、ReLU活性化関数を適用

    x_num = BatchNormalization()(x_num)  ### バッチ正規化を適用

    x_num = Dropout(0.2)(x_num)  ### ドロップアウトを適用

    x_num = Dense(5, activation="relu")(x_num)  ### 5個のニューロンを持つ全結合層を追加し、ReLU活性化関数を適用

    x_num = BatchNormalization()(x_num)  ### バッチ正規化を適用

    x_num = Dropout(0.1)(x_num)  ### ドロップアウトを適用

    out = Dense(1, activation="sigmoid")(x_num)  ### 出力層を定義し、シグモイド活性化関数を適用

    model = Model(inputs=input_num,  ### モデルを作成し、入力と出力を指定
                  outputs=out,
                 )

    model.compile(  ### モデルをコンパイルし、損失関数、最適化アルゴリズム、評価メトリクスを指定
        optimizer="Adam",
        loss="binary_crossentropy",
        metrics=["binary_crossentropy"],
    )

    return model  ### 作成されたモデルを返す

model = create_model()  ### create_model()関数を呼び出してモデルを作成

model.summary()  ### 作成されたモデルの構造を要約

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_28 (Dense)            (None, 10)                90        
                                                                 
 batch_normalization_21 (Ba  (None, 10)                40        
 tchNormalization)                                               
                                                                 
 dropout_21 (Dropout)        (None, 10)                0         
                                                                 
 dense_29 (Dense)            (None, 10)                110       
                                                                 
 batch_normalization_22 (Ba  (None, 10)                40        
 tchNormalization)                                         

#### モデル学習

In [48]:
seed_everything(seed=123)  # 乱数ジェネレータのシードを固定するための関数呼び出し

model = create_model()  # ニューラルネットワークモデルを作成する関数を呼び出してモデルを作成

model.fit(  # モデルを訓練するメソッドの呼び出し
    x=x_tr,  # 訓練データの入力
    y=y_tr,  # 訓練データの出力
    validation_data=(x_va, y_va),  # 検証データの入力と出力
    batch_size=8,  # バッチサイズを8に設定
    epochs=10000,  # エポック数を10000に設定
    callbacks=[  # コールバック関数のリスト
        ModelCheckpoint(  # ベストなモデルの重みを保存するコールバック
            filepath="model_keras.weights.h5",  # ファイルパス
            monitor="val_loss",  # 監視する指標は検証データの損失
            mode="min",  # 目標は最小化
            verbose=1,  # ログを表示
            save_best_only=True,  # 最も良いモデルのみ保存
            save_weights_only=True,  # 重みのみ保存
        ),
        EarlyStopping(  # 訓練を早期に停止するコールバック
            monitor="val_loss",  # 監視する指標は検証データの損失
            mode="min",  # 目標は最小化
            min_delta=0,  # 損失の改善がこの値より小さくなった場合に停止しない
            patience=10,  # 損失が改善されないエポック数の許容範囲
            verbose=1,  # ログを表示
            restore_best_weights=True,  # 最も良いモデルの重みを復元
        ),
        ReduceLROnPlateau(  # 学習率を減少させるコールバック
            monitor="val_loss",  # 監視する指標は検証データの損失
            mode="min",  # 目標は最小化
            factor=0.1,  # 学習率を減少させる因子
            patience=5,  # 損失が改善されないエポック数の許容範囲
            verbose=1,  # ログを表示
        ),
    ],
    verbose=1,  # ログを表示
)

Epoch 1/10000
16497/16504 [============================>.] - ETA: 0s - loss: 0.4277 - binary_crossentropy: 0.4277
Epoch 1: val_loss improved from inf to 0.36192, saving model to model_keras.weights.h5
16504/16504 [==============================] - 74s 4ms/step - loss: 0.4277 - binary_crossentropy: 0.4277 - val_loss: 0.3619 - val_binary_crossentropy: 0.3619 - lr: 0.0010
Epoch 2/10000
16490/16504 [============================>.] - ETA: 0s - loss: 0.3921 - binary_crossentropy: 0.3921
Epoch 2: val_loss improved from 0.36192 to 0.35242, saving model to model_keras.weights.h5
16504/16504 [==============================] - 73s 4ms/step - loss: 0.3921 - binary_crossentropy: 0.3921 - val_loss: 0.3524 - val_binary_crossentropy: 0.3524 - lr: 0.0010
Epoch 3/10000
16491/16504 [============================>.] - ETA: 0s - loss: 0.3866 - binary_crossentropy: 0.3866
Epoch 3: val_loss improved from 0.35242 to 0.35199, saving model to model_keras.weights.h5
16504/16504 [==============================] - 

#### モデルの評価

In [ ]:
y_va_pred = model.predict(x_va, batch_size=8, verbose=1)  # 検証データセット (x_va) 上での予測を行う。バッチサイズは8、予測の進行状況を表示する。

print("accuracy: {:.4f}".format(  # 精度を表示するための文字列を準備し、
    accuracy_score(  # 精度を計算する
        y_va,  # 実際のラベル
        np.where(y_va_pred>=0.5, 1,0)  # 予測された確率が0.5以上の場合は1に、それ以外の場合は0に変換した予測値
    )
))  # 計算された精度を表示する

4126/4126 [==============================] - 7s 2ms/step
accuracy: 0.8492


推論用データセットを作成

In [58]:
# ファイル読み込み
df_test = pd.read_csv("/content/drive/MyDrive/kaggle実践/銀行解約データセットを使用した二項分類/Bank Churn/test.csv")

#特徴量エンジニアリング
df_test["NumOfProducts_/_Age"] = df_test["NumOfProducts"] / df_test["Age"]
df_test["Balance_-_NumOfProducts"] = df_test["Balance"] - df_test["NumOfProducts"]
df_test["mean_NumOfProducts_by_HasCrCard"] = df_test.groupby("NumOfProducts")["HasCrCard"].transform("mean")

# データセット作成
x_test = df_test[["IsActiveMember","EstimatedSalary","Age","NumOfProducts","NumOfProducts_/_Age","Balance_-_NumOfProducts","mean_NumOfProducts_by_HasCrCard"]]
id_test = df_test[["id"]]

数値データの正規化

In [57]:
### "下の各特徴量の列の値を最小値で引き、その結果を"下の特徴量の列の最大値と最小値の差で割ることで、最小-最大スケーリングを実行
x_test["IsActiveMember"] = (x_test["IsActiveMember"] - x_test["IsActiveMember"].min()) / (x_test["IsActiveMember"].max() - x_test["IsActiveMember"].min())

x_test["EstimatedSalary"] = (x_test["EstimatedSalary"] - x_test["EstimatedSalary"].min()) / (x_test["EstimatedSalary"].max() - x_test["EstimatedSalary"].min())

x_test["Age"] = (x_test["Age"] - x_test["Age"].min()) / (x_test["Age"].max() - x_test["Age"].min())

x_test["NumOfProducts"] = (x_test["NumOfProducts"] - x_test["NumOfProducts"].min()) / (x_test["NumOfProducts"].max() - x_test["NumOfProducts"].min())
x_test["NumOfProducts_/_Age"] = (x_test["NumOfProducts_/_Age"] - x_test["NumOfProducts_/_Age"].min()) / (x_test["NumOfProducts_/_Age"].max() - x_test["NumOfProducts_/_Age"].min())
x_test["Balance_-_NumOfProducts"] = (x_test["Balance_-_NumOfProducts"] - x_test["Balance_-_NumOfProducts"].min()) / (x_test["Balance_-_NumOfProducts"].max() - x_test["Balance_-_NumOfProducts"].min())
x_test["mean_NumOfProducts_by_HasCrCard"] = (x_test["mean_NumOfProducts_by_HasCrCard"] - x_test["mean_NumOfProducts_by_HasCrCard"].min()) / (x_test["mean_NumOfProducts_by_HasCrCard"].max() - x_test["mean_NumOfProducts_by_HasCrCard"].min())



テストデータ予測

In [55]:
y_test_pred = model.predict(x_test)


3439/3439 [==============================] - 7s 2ms/step


提出ファイルの作成

In [56]:
df_submit = pd.DataFrame({"id": id_test["id"], "Exited": y_test_pred[:, 0]})
display(df_submit.head(5))
df_submit.to_csv("/content/drive/MyDrive/kaggle実践/銀行解約データセットを使用した二項分類/submission_tensorflow1.csv", index=None)

,id,Exited
0,165034,0.017303
1,165035,0.744566
2,165036,0.063497
3,165037,0.233344
4,165038,0.211090


### ニューラルネットワークの適用例：②埋め込み層ありのネットワークモデル
#### ファイルの読み込みとデータセット作成

In [ ]:
## ファイル読み込み
df_train = pd.read_csv("/content/drive/MyDrive/kaggle実践/銀行解約データセットを使用した二項分類/Bank Churn/train.csv")


# データセット作成
x_train = df_train[["IsActiveMember","EstimatedSalary","Age","NumOfProducts","Gender"]]
y_train = df_train[["Exited"]]


#### 数値データの前処理

In [ ]:
# 正規化
for col in ["IsActiveMember", "EstimatedSalary","Age","NumOfProducts"]:
    value_min = x_train[col].min()
    value_max = x_train[col].max()
    x_train[col] = (x_train[col] - value_min) / (value_max - value_min)

#### カテゴリ変数の前処理

In [ ]:
# 欠損値補間は欠損値なしのため未実施


# label-encoding：ラベルエンコーディングを行うための処理
le = LabelEncoder()  # ラベルエンコーダーのインスタンスを生成
le.fit(x_train[["Gender"]])  # "Surname" 列の値を用いてラベルエンコーダーを適合させる
x_train["Gender"] = le.transform(x_train["Gender"])  # "Surname" 列の各値をラベルエンコーダーを使用して数値に変換する

print(le.classes_)  # ラベルエンコーダーが学習したクラス（カテゴリ）のリストを表示する
print("count:", len(le.classes_))  # 学習したクラス（カテゴリ）の数を表示する

['Female' 'Male']
count: 2


#### 学習データと検証データの分離

In [ ]:
x_train_num, x_train_cat = x_train[["IsActiveMember", "EstimatedSalary","Age","NumOfProducts"]], x_train[["Gender"]]

x_num_tr, x_num_va, x_cat_tr, x_cat_va, y_tr, y_va = \
   train_test_split(x_train_num, x_train_cat, y_train, test_size=0.2, stratify=y_train, random_state=123)
print(x_num_tr.shape, x_num_va.shape, x_cat_tr.shape, x_cat_va.shape, y_tr.shape, y_va.shape)

(132027, 4) (33007, 4) (132027, 1) (33007, 1) (132027, 1) (33007, 1)


#### モデル定義

In [ ]:
def create_model_embedding():  # 埋め込みを使用したモデルを作成する関数の定義

    ################# num
    input_num = Input(shape=(4,))  # 数値データ用の入力層を定義し、形状を指定する

    layer_num = Dense(10, activation="relu")(input_num)  # 全結合層を定義し、活性化関数をReLUとして適用する
    layer_num = BatchNormalization()(layer_num)  # バッチ正規化を適用する
    layer_num = Dropout(0.2)(layer_num)  # ドロップアウトを適用する
    layer_num = Dense(10, activation="relu")(layer_num)  # もう一つの全結合層を定義し、活性化関数をReLUとして適用する

    ################# cat
    input_cat = Input(shape=(1,))  # カテゴリカルデータ用の入力層を定義し、形状を指定する

    layer_cat = input_cat[:, 0]  # カテゴリカルデータの最初の要素を取得する
    layer_cat = Embedding(input_dim=148, output_dim=74)(layer_cat)  # 埋め込み層を定義し、カテゴリ数と埋め込み次元を指定する
    layer_cat = Dropout(0.2)(layer_cat)  # ドロップアウトを適用する
    layer_cat = Flatten()(layer_cat)  # フラット化する

    ################# concat
    hidden_layer = Concatenate()([layer_num, layer_cat])  # 数値データとカテゴリカルデータを結合する
    hidden_layer = Dense(50, activation="relu")(hidden_layer)  # 全結合層を定義し、活性化関数をReLUとして適用する
    hidden_layer = BatchNormalization()(hidden_layer)  # バッチ正規化を適用する
    hidden_layer = Dropout(0.1)(hidden_layer)  # ドロップアウトを適用する
    hidden_layer = Dense(20, activation="relu")(hidden_layer)  # もう一つの全結合層を定義し、活性化関数をReLUとして適用する
    hidden_layer = BatchNormalization()(hidden_layer)  # バッチ正規化を適用する
    hidden_layer = Dropout(0.1)(hidden_layer)  # ドロップアウトを適用する

    output_layer = Dense(1, activation="sigmoid")(hidden_layer)  # 出力層を定義し、活性化関数をシグモイドとして適用する

    model = Model(inputs=[input_num, input_cat],  # 入力層を指定し、
                  outputs=output_layer,  # 出力層を指定する
                 )

    model.compile(  # モデルをコンパイルする
        optimizer="Adam",  # 最適化アルゴリズムを指定する
        loss="binary_crossentropy",  # 損失関数を指定する
        metrics=["binary_crossentropy"],  # 評価指標を指定する
    )

    return model  # 定義したモデルを返す

model = create_model_embedding()  # 上記の関数を使用してモデルを作成する
model.summary()  # モデルの概要を表示する

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 4)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 dense (Dense)               (None, 10)                   50        ['input_1[0][0]']             
                                                                                                  
 tf.__operators__.getitem (  (None,)                      0         ['input_2[0][0]']             
 SlicingOpLambda)                                                                             

#### モデルの学習

In [ ]:
model = create_model_embedding()  # 埋め込みを含むモデルを作成する関数を呼び出す
model.fit(x=[x_num_tr, x_cat_tr],  # 数値特徴とカテゴリカル特徴のトレーニングデータを指定
          y=y_tr,  # トレーニングデータの目標値を指定
          validation_data=([x_num_va, x_cat_va], y_va),  # 検証データを指定
          batch_size=8,  # ミニバッチのサイズを指定
          epochs=10000,  # エポック数を指定
          callbacks=[  # コールバック関数のリストを指定
              ModelCheckpoint(filepath="model_keras_embedding.weights.h5",  # ベストのモデルの重みを保存するファイルパスを指定
                              monitor="val_loss",  # 監視する指標を指定
                              mode="min",  # 監視する指標の最小化を指定
                              verbose=1,  # 進捗情報を表示するかどうかを指定
                              save_best_only=True,  # 最良のモデルのみ保存するかどうかを指定
                              save_weights_only=True),  # 重みのみを保存するかどうかを指定
              EarlyStopping(monitor="val_loss",  # 監視する指標を指定
                            mode="min",  # 監視する指標の最小化を指定
                            min_delta=0,  # 早期停止の閾値を指定
                            patience=10,  # 指標が改善されないエポック数の許容範囲を指定
                            verbose=1,  # 進捗情報を表示するかどうかを指定
                            restore_best_weights=True),  # 最良の重みを復元するかどうかを指定
              ReduceLROnPlateau(monitor="val_loss",  # 監視する指標を指定
                                mode="min",  # 監視する指標の最小化を指定
                                factor=0.1,  # 学習率を減少させる因子を指定
                                patience=5,  # 指標が改善されないエポック数の許容範囲を指定
                                verbose=1),  # 進捗情報を表示するかどうかを指定
          ],
          verbose=1,  # トレーニングの進行状況を表示する方法を指定
         )

Epoch 1/10000
16491/16504 [============================>.] - ETA: 0s - loss: 0.4145 - binary_crossentropy: 0.4145
Epoch 1: val_loss improved from inf to 0.36932, saving model to model_keras_embedding.weights.h5
16504/16504 [==============================] - 77s 4ms/step - loss: 0.4144 - binary_crossentropy: 0.4144 - val_loss: 0.3693 - val_binary_crossentropy: 0.3693 - lr: 0.0010
Epoch 2/10000
16498/16504 [============================>.] - ETA: 0s - loss: 0.3889 - binary_crossentropy: 0.3889
Epoch 2: val_loss did not improve from 0.36932
16504/16504 [==============================] - 64s 4ms/step - loss: 0.3889 - binary_crossentropy: 0.3889 - val_loss: 0.3835 - val_binary_crossentropy: 0.3835 - lr: 0.0010
Epoch 3/10000
16496/16504 [============================>.] - ETA: 0s - loss: 0.3833 - binary_crossentropy: 0.3833
Epoch 3: val_loss did not improve from 0.36932
16504/16504 [==============================] - 86s 5ms/step - loss: 0.3833 - binary_crossentropy: 0.3833 - val_loss: 0.3805 -

#### モデル評価

In [ ]:
y_va_pred = model.predict([x_num_va, x_cat_va], batch_size=8, verbose=1)  # 検証データに対する予測を行う

print("accuracy: {:.4f}".format(accuracy_score(y_va, np.where(y_va_pred>=0.5,1,0))))  # 予測結果の精度を評価して出力する

4126/4126 [==============================] - 9s 2ms/step
accuracy: 0.8494


推論用データセット作成

In [ ]:
## ファイル読み込み
df_test = pd.read_csv("/content/drive/MyDrive/kaggle実践/銀行解約データセットを使用した二項分類/Bank Churn/test.csv")


# データセット作成
x_num_test = x_test[["IsActiveMember", "EstimatedSalary", "Age", "NumOfProducts"]]
x_cat_test = x_test[["Gender"]]

id_test = df_test[["id"]]



数値データの前処理

In [ ]:
# 正規化
for col in ["IsActiveMember", "EstimatedSalary","Age","NumOfProducts"]:
    value_min = x_test[col].min()
    value_max = x_test[col].max()
    x_test[col] = (x_test[col] - value_min) / (value_max - value_min)

カテゴリ変数の前処理

In [ ]:

# label-encoding：ラベルエンコーディングを行うための処理
le = LabelEncoder()  # ラベルエンコーダーのインスタンスを生成
le.fit(x_test[["Gender"]])  # "Surname" 列の値を用いてラベルエンコーダーを適合させる
x_test["Gender"] = le.transform(x_test["Gender"])  # "Surname" 列の各値をラベルエンコーダーを使用して数値に変換する

print(le.classes_)  # ラベルエンコーダーが学習したクラス（カテゴリ）のリストを表示する
print("count:", len(le.classes_))  # 学習したクラス（カテゴリ）の数を表示する

[0 1]
count: 2


データ予測

In [ ]:
y_test_pred = model.predict([x_num_test, x_cat_test])

3439/3439 [==============================] - 9s 3ms/step


提出ファイル作成

In [ ]:
df_submit = pd.DataFrame({"id": id_test["id"], "Exited": y_test_pred[:, 0]})
display(df_submit.head(5))
df_submit.to_csv("/content/drive/MyDrive/kaggle実践/銀行解約データセットを使用した二項分類/submission_tensorflow2.csv", index=None)

,id,Exited
0,165034,0.015464
1,165035,0.780960
2,165036,0.073873
3,165037,0.226839
4,165038,0.259119
